# Imports

In [2]:
import pandas as pd
import numpy as np
import os
import pickle

# import distance calculation function from module
import haversine_formula as hf

# Load Data
This data comes from [Divvy](https://www.divvybikes.com/system-data), where I downloaded all the csv files associated with 2020.
First, we will load all the csv files as dataframes and concat them into a single dataframe.

In [4]:
dfs = {}
for file in [f for f in os.listdir('data/') if f.endswith('csv')]:
    # load csv files
    dfs[file]=pd.read_csv('data/'+file)
    print(file, '\t', dfs[file].shape)

#concat csvs into 1 big dataframe
df= pd.concat(dfs.values())
df = df.sort_values(by=['started_at']).reset_index(drop=True)
print(df.shape)

202004-divvy-tripdata.csv 	 (84776, 13)
202005-divvy-tripdata.csv 	 (200274, 13)
202006-divvy-tripdata.csv 	 (343005, 13)
Divvy_Trips_2020_Q1.csv 	 (426887, 13)
(1054942, 13)


# Prepare Data
We're going to parse and process the data first because we will use the derived information to clean the dataframe afterwards.

## Parse Datetime

In [5]:
def parse_datetime(df):
    '''
    df [dataframe] = Divvy dataframe with string timestamp columns
    Function to convert string timestamp columns to datetime columns and parse out week number.
    df [dataframe] = Divvy dataframe with a parsed week column
    '''
    # convert string date time to a datetime object
    df['started_at'] = pd.to_datetime(df['started_at'])
    df['ended_at'] = pd.to_datetime(df['ended_at'])
    # parse out the numerical week from the s-tart datetime
    df['start_week'] = df['started_at'].dt.week
    return df

## Calculate Duration

In [9]:
def get_minutes(ended_at, started_at):
    '''
    ended_at [datetime] = time of bicycle return
    started_at [datetime] = time of bicycle rental
    Function to calculate duration in minutes between two datetime objects.
    minutes [float] = duration in minutes
    '''
    minutes = ((ended_at - started_at).total_seconds())/60
    return minutes

In [5]:
def add_duration(df):
    '''
    df [dataframe] = Divvy dataframe with datetime columns
    Function to apply the get_minutes function to all the rows.
    df [dataframe] = Divvy dataframe with a duration column
    '''
    # applies the get_minutes function to every row of the dataframe
    df['duration_minutes'] = df.apply(lambda row: get_minutes(row['ended_at'], row['started_at']), axis = 1)
    return df

## Calculate Distance

In [7]:
def add_trip_distance(df):
    '''
    df [dataframe] = Divvy dataframe with starting latitude, starting longitude, ending latitude, and ending longitude columns
    Function to apply lat_lng_distance fuction from the haversine_formula.py module to all the rows.
    df [dataframe] = Divvy dataframe with a trip distance column
    '''
    # using the haversine fomula we calcuate the miles between where the trip started and where the trip ended
    df['distance'] = df.apply(lambda row: hf.lat_lng_distance(row['start_lat'], row['start_lng'], 
                                                         row['end_lat'], row['end_lng']), axis = 1) 
    return df

## Prepare Dataframe

In [8]:
def prepare_dataframe(df):
    '''
    df [dataframe] = Divvy dataframe with string timestamp columns, and
                     starting latitude, starting longitude, ending latitude, and ending longitude columns
    Function to apply all the other data parsing, processing and preparation functions.
    df [dataframe] = Divvy dataframe with a parsed week column, a duration column, and a trip distance column
    '''
    # runs all the parsing and processing functions
    print('Before: ', df.shape)
    df = parse_datetime(df)
    df = add_duration(df)
    df = add_trip_distance(df)
    print('After: ', df.shape)
    return df

In [9]:
df = prepare_dataframe(df)

Before:  (1054942, 13)
After:  (1054942, 16)


# Clean Data

## Forgot to Return
According to Divvy's FAQs, if a bicycle has been missing for longer than 24 hours, the account holder that took out the bicycle will be charged $1,200 fee (plus tax). As such, I used 24 hours to remove rides with outlier durations. 

In [12]:
def remove_forgot_to_returns(df):
    '''
    df [dataframe] = Divvy dataframe with a duration column
    Function to remove rows with trips longer than 24 hours in duration.
    cleaner_df [dataframe] = Divvy dataframe with rides longer than 24 hours filtered out
    '''
    # Few rides that were longer than 24 hours were pulling the duration mean outward, so we will remove them as outliers
    cleaner_df = df[df['duration_minutes']<(24*60)] # rides that are 24 hours or longer
    return cleaner_df

## Instant Returns

In [11]:
def remove_instant_returns(df):
    '''
    df [dataframe] = Divvy dataframe with a duration column and a trip distance column
    Function to remove trips less than 1 minute in duration and with no distance.
    cleaner_df [dataframe] = Divvy dataframe without rides that were less than 1 minute and 0 distance
    '''
    # The numerous instant returns, rides that started but instantly ended, 
    # were pulling the mean distance and duration downward.
    # because the were skewing the behavior of average riders, I removed these as outliers.
    cleaner_df = df[~((df['duration_minutes']<1) & (df['distance']==0))]
    return cleaner_df

## Rides in Progress

In [12]:
def remove_rides_in_progress(df):
    '''
    df [dataframe] = Divvy dataframe with end station name column 
    Function to remove rows with nulls indicating that the trips never ended.
    cleaner_df [dataframe] = Divvy dataframe without rows with null end station names
    '''
    # This function will remove all the rows with nulls
    cleaner_df = df[~df['end_station_name'].isnull()]
    return cleaner_df

## Last Week

In [13]:
def remove_last_week(df):
    '''
    df [dataframe] = Divvy dataframe with a parsed week column
    Function to remove rows with the last week number.
    cleaner_df [dataframe] = Divvy dataframe with only data points from complete weeks
    '''
    # Since we are analyzing our data in within week groups, 
    # and incomplete week's data will not be an accurate reflection of that week. 
    # There are only 2 days in the last week of June. 
    # So we have to remove the last week as it is incomplete. 
    cleaner_df = df[df['start_week'] != sorted(df['start_week'].unique())[-1]]
    return cleaner_df

## Clean dataframe

In [ ]:
df [dataframe] = Divvy dataframe with a duration column and a trip distance column with end station name column with a parsed week column

cleaner_df [dataframe] = Divvy dataframe with rides longer than 24 hours filtered out, without rides that were less than 1 minute and 0 distance  without rows with null end station nameswith only data points from complete weeks

In [14]:
def clean_dataframe(df):
    '''
    df [dataframe] = Divvy dataframe with a parsed week column, a duration column,
                    a trip distance column, and end station name column
    Function to apply all the other dataframe cleaning functions.
    df [dataframe] = Divvy dataframe without rides longer than 24 hours, rides less thant 1 minute with 0 distance, 
                    rides without end stations, and rides from the last week. 
    '''
    # running all the cleaning functions
    print('Before: ', df.shape)
    df = remove_forgot_to_returns(df)
    df = remove_instant_returns(df)
    df = remove_rides_in_progress(df)
    df = remove_last_week(df)
    print('After: ', df.shape)
    return df

In [15]:
df = clean_dataframe(df)

Before:  (1054942, 16)
After:  (1015635, 16)


# Save Dataframe

In [16]:
#create pickle file of cleaned dataframe
with open('data/divvy_trips.pkl', 'wb') as picklefile:
     pickle.dump(df, picklefile)